# Proyecto: Búsqueda en Árbol — *Peg Solitaire* (Come Solo)

Autores: **Juan Sebastian Abarca Hernandez y Juan Antonio Trenado Ballesteros**  
Fecha: 24 de Septiembre de 2025

**Repositorio base (fork de `baile`)**: https://github.com/SebastianAbarca25/baileProyecto1

Este cuaderno documenta la **definición formal del problema**, los **experimentos** con búsqueda **no informada** (BFS, DFS) y **A\***, la **comparación de resultados** y las **conclusiones**.


## Instrucciones rápidas
1. Ejecuta todas las celdas en orden. 
2. Para **probar otras configuraciones** (por ejemplo, tableros alternos o profundidad límite para DFS), modifica los parámetros en la sección de *Experimentos*.
3. Al final hay una celda para **imprimir la secuencia de movimientos** y **reconstruir la solución**.

### Citas y contexto
- Librería base: `baile` — *Baisic Artificial Intelligence Library for Education* (GitHub: `https://github.com/kyriox/baile`).
- Dominio: *Peg Solitaire* (tablero inglés de 33 posiciones, salto ortogonal, se retira la ficha sobre la que se salta; objetivo: una sola ficha, idealmente en el centro).


## 1) Definición formal del problema
**Estado**: representamos el tablero como una **matriz 7×7**, donde:
- `-1` = fuera del tablero (celdas no válidas),
- `0`  = vacío,
- `1`  = hay ficha.

Usamos el tablero inglés estándar (33 casillas válidas). El estado inicial tiene **todas las casillas válidas ocupadas salvo el centro**.

**Operador/sucesor**: un movimiento válido es un **salto ortogonal** (arriba/abajo/izquierda/derecha) de una ficha `1` sobre otra `1` hacia una casilla vacía `0`, retirando la ficha intermedia. Formalmente, para una dirección `d∈{(±1,0),(0,±1)}` si `s[i,j]=1`, `s[i+di,j+dj]=1` y `s[i+2di,j+2dj]=0`, entonces se genera un nuevo estado con `s[i,j]=0`, `s[i+di,j+dj]=0`, `s[i+2di,j+2dj]=1`.

**Función de meta**: `goal(s)` es verdadero si el **número de fichas** en `s` es `1`. (Podemos añadir la variante *meta central* que además exige que la ficha esté en el centro `(3,3)`.)

**Heurística (admisible)**:  
Usamos `h₁(s) = max(0, pegs(s) - 1)`.
Justificación: cada movimiento **siempre elimina exactamente una ficha**, por lo que el **número mínimo de movimientos restantes** es al menos `pegs − 1`. Es una cota inferior y, por tanto, **admisible**.

Adicionalmente incluimos una **heurística informativa no admisible** opcional para análisis comparativo: `h₂(s) = (pegs(s)-1) + λ·dist_min_al_centro(s)`, con `λ>0`, para influir el sesgo hacia el centro (solo para análisis, no garantiza optimalidad).


In [ ]:
from src.peg_solitaire_problem import PegSolitaireProblem, run_experiments

# Librerías de apoyo
import pandas as pd   # Para mostrar tablas bonitas
import matplotlib.pyplot as plt   # (opcional) para gráficas
import time
import collections

In [ ]:
# Utilidades de tablero Peg Solitaire (7x7 inglés)
from typing import List, Tuple, Iterable, Optional, Dict, Any

Board = List[List[int]]  # -1 fuera, 0 vacío, 1 ficha
Vec = Tuple[int,int]
Move = Tuple[Tuple[int,int], Tuple[int,int]]  # (origen)->(destino)

DIRS: List[Vec] = [(1,0),(-1,0),(0,1),(0,-1)]

def english_board() -> Board:
    # Plantilla del tablero inglés estándar (7x7 con 33 casillas válidas)
    O, X = -1, 1
    row = lambda a: [1 if c=='X' else (-1 if c=='O' else 0) for c in a]
    # Usamos 'X' como casilla válida inicial con ficha; luego vaciamos el centro
    layout = [
        ['O','O','X','X','X','O','O'],
        ['O','O','X','X','X','O','O'],
        ['X','X','X','X','X','X','X'],
        ['X','X','X','0','X','X','X'],  # marcaremos '0' para vacío inicial
        ['X','X','X','X','X','X','X'],
        ['O','O','X','X','X','O','O'],
        ['O','O','X','X','X','O','O'],
    ]
    board: Board = []
    for r in layout:
        br = []
        for c in r:
            if c=='O': br.append(-1)
            elif c=='0': br.append(0)
            else: br.append(1)
        board.append(br)
    return board

def clone(b: Board) -> Board:
    return [row[:] for row in b]

def in_bounds(b: Board, i:int, j:int) -> bool:
    return 0 <= i < 7 and 0 <= j < 7 and b[i][j] != -1

def count_pegs(b: Board) -> int:
    return sum(1 for i in range(7) for j in range(7) if b[i][j]==1)

def moves(b: Board) -> Iterable[Move]:
    for i in range(7):
        for j in range(7):
            if b[i][j] != 1: continue
            for di,dj in DIRS:
                i1,j1 = i+di, j+dj
                i2,j2 = i+2*di, j+2*dj
                if in_bounds(b,i2,j2) and b[i1][j1]==1 and b[i2][j2]==0:
                    yield ((i,j),(i2,j2))

def apply_move(b: Board, m: Move) -> Board:
    (i,j),(i2,j2) = m
    di,dj = (i2-i)//2, (j2-j)//2
    nb = clone(b)
    nb[i][j] = 0
    nb[i+di][j+dj] = 0
    nb[i2][j2] = 1
    return nb

def is_goal_one(b: Board) -> bool:
    return count_pegs(b) == 1

def is_goal_one_center(b: Board) -> bool:
    return count_pegs(b) == 1 and b[3][3]==1

def h_pegs(b: Board) -> int:
    return max(0, count_pegs(b) - 1)

def dist_to_center_min(b: Board) -> int:
    # distancia de Manhattan mínima de alguna ficha al centro (3,3)
    D = []
    for i in range(7):
        for j in range(7):
            if b[i][j]==1: D.append(abs(i-3)+abs(j-3))
    return min(D) if D else 0

def h_non_admissible(b: Board, lam: float = 0.25) -> float:
    return (count_pegs(b) - 1) + lam * dist_to_center_min(b)

def board_str(b: Board) -> str:
    s=''
    for i in range(7):
        row=[]
        for j in range(7):
            row.append({-1:' ',0:'.',1:'o'}[b[i][j]])
        s += ' '.join(row)+"\n"
    return s

start = english_board()
print('Estado inicial:')
print(board_str(start))


Estado inicial:
    o o o    
    o o o    
o o o o o o o
o o o . o o o
o o o o o o o
    o o o    
    o o o    



In [ ]:
from heapq import heappush, heappop
from dataclasses import dataclass

@dataclass
class Node:
    board: Board
    g: int
    move: Optional[Move]
    parent: Optional[int]  # índice al vector de nodos para reconstrucción

def reconstruct_path(nodes: list[Node], idx: int) -> list[Move]:
    path = []
    while idx is not None:
        n = nodes[idx]
        if n.move is not None:
            path.append(n.move)
        idx = n.parent
    path.reverse()
    return path

def bfs(start: Board, goal_fn=is_goal_one):
    t0=time.time();
    frontier=collections.deque([0])
    nodes=[Node(start,0,None,None)]
    seen={}
    expanded=0
    seen[str(start)] = 0
    while frontier:
        idx=frontier.popleft()
        n=nodes[idx]
        expanded+=1
        if goal_fn(n.board):
            return {
                'found':True,
                'path':reconstruct_path(nodes, idx),
                'time':time.time()-t0,
                'expanded':expanded,
                'depth':n.g
            }
        for m in moves(n.board):
            nb = apply_move(n.board, m)
            key = str(nb)
            if key not in seen:
                seen[key]=1
                nodes.append(Node(nb,n.g+1,m,idx))
                frontier.append(len(nodes)-1)
    return {'found':False,'time':time.time()-t0,'expanded':expanded}

def dfs(start: Board, goal_fn=is_goal_one, depth_limit: int = 64):
    t0=time.time();
    nodes=[Node(start,0,None,None)]
    expanded=0
    best=None
    sys.setrecursionlimit(10000)
    seen=set()

    def rec(idx:int):
        nonlocal expanded, best
        n=nodes[idx]
        expanded+=1
        if goal_fn(n.board):
            best = reconstruct_path(nodes, idx)
            return True
        if n.g >= depth_limit:
            return False
        key=str(n.board)
        seen.add(key)
        for m in moves(n.board):
            nb=apply_move(n.board,m)
            k=str(nb)
            if k in seen: 
                continue
            nodes.append(Node(nb,n.g+1,m,idx))
            if rec(len(nodes)-1):
                return True
        seen.discard(key)
        return False

    found = rec(0)
    return {
        'found':found,
        'path':best if found else None,
        'time':time.time()-t0,
        'expanded':expanded,
        'depth':len(best) if best else None
    }

def astar(start: Board, goal_fn=is_goal_one, heuristic=h_pegs):
    t0=time.time();
    nodes=[Node(start,0,None,None)]
    expanded=0
    openpq=[]
    heappush(openpq, (heuristic(start), 0))
    best_g={str(start):0}
    while openpq:
        _, idx = heappop(openpq)
        n=nodes[idx]
        expanded+=1
        if goal_fn(n.board):
            return {
                'found':True,
                'path':reconstruct_path(nodes, idx),
                'time':time.time()-t0,
                'expanded':expanded,
                'depth':n.g
            }
        for m in moves(n.board):
            nb = apply_move(n.board,m)
            g2 = n.g + 1
            k = str(nb)
            if k not in best_g or g2 < best_g[k]:
                best_g[k]=g2
                nodes.append(Node(nb,g2,m,idx))
                f = g2 + heuristic(nb)
                heappush(openpq,(f,len(nodes)-1))
    return {'found':False,'time':time.time()-t0,'expanded':expanded}


## 2) Experimentos: BFS, DFS y A*
Parámetros por defecto: meta = **una ficha** en cualquier lugar. Puedes activar la variante *meta central* más abajo.


In [ ]:
import time
import collections

start = english_board()
print('Pegs iniciales:', count_pegs(start))
print(board_str(start))

results = {}
results['BFS']  = bfs(start, goal_fn=is_goal_one)
results['DFS']  = dfs(start, goal_fn=is_goal_one, depth_limit=70)
results['A* (h_pegs)'] = astar(start, goal_fn=is_goal_one, heuristic=h_pegs)
results


Pegs iniciales: 32
    o o o    
    o o o    
o o o o o o o
o o o . o o o
o o o o o o o
    o o o    
    o o o    



In [ ]:
# Variante: exigir meta en el CENTRO y comparar A* con heurística no admisible (solo para análisis)
results_center = {}
results_center['A* (h_pegs) meta centro'] = astar(start, goal_fn=is_goal_one_center, heuristic=h_pegs)
results_center['A* (no admisible) meta centro'] = astar(start, goal_fn=is_goal_one_center, heuristic=lambda b: h_non_admissible(b, 0.25))
results_center


NameError: name 'time' is not defined

In [ ]:
import pandas as pd
def tabulate(resdict):
    rows=[]
    for k,v in resdict.items():
        rows.append({
            'Algoritmo': k,
            'Encontró?': v.get('found',False),
            'Tiempo (s)': round(v.get('time',float('nan')),4),
            'Nodos expandidos': v.get('expanded',None),
            'Long. solución (movs)': v.get('depth',None),
        })
    return pd.DataFrame(rows)

print("Resultados — meta libre")
display(df1)

print("\nResultados — meta centro")
display(df2)

import caas_jupyter_tools
caas_jupyter_tools.display_dataframe_to_user('Resultados — meta libre', df1)
caas_jupyter_tools.display_dataframe_to_user('Resultados — meta centro', df2)
df1, df2


ModuleNotFoundError: No module named 'caas_jupyter_tools'

## 3) Reconstrucción de la solución y visualización paso a paso


In [ ]:
def show_path(res):
    if not res.get('found'):
        print("No se encontró solución.")
        return
    b=english_board()
    print("Estado inicial:")
    print(board_str(b))
    for t,m in enumerate(res['path'], start=1):
        b = apply_move(b,m)
        print(f'Paso {t}: {m}')
        print(board_str(b))

#Solucion de A
show_path(results['A* (h_pegs)'])

KeyError: 'A* (h_pegs)'

## 4) Conclusiones y observaciones

